In [73]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestNeighbors
from joblib import dump, load



In [6]:
dataframe = pd.read_csv('./data/akc-data-latest-v3.csv')
dataframe.head()

,breed,description,temperament,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,...,trainability_category,demeanor_value,demeanor_category,temperament_category,height,weight,expectancy,height_category,weight_category,expectancy_category
0,Affenpinscher,The Affen’s apish look has been described many...,"Confident, Famously Funny, Fearless",148,22.86,29.21,3.175147,4.535924,12.0,15.0,...,Easy Training,1.0,Outgoing,Confident,26.035,3.855535,13.5,Short,Light,Long
1,Afghan Hound,"The Afghan Hound is an ancient breed, his whol...","Dignified, Profoundly Loyal, Aristocratic",113,63.50,68.58,22.679619,27.215542,12.0,15.0,...,May be Stubborn,0.2,Aloof/Wary,Loyal,66.040,24.947580,13.5,Tall,Medium,Long
2,Airedale Terrier,The Airedale Terrier is the largest of all ter...,"Friendly, Clever, Courageous",60,58.42,58.42,22.679619,31.751466,11.0,14.0,...,Eager to Please,0.8,Friendly,Amiable,58.420,27.215542,12.5,Tall,Medium,Long
3,Akita,"Akitas are burly, heavy-boned spitz-type dogs ...","Courageous, Dignified, Profoundly Loyal",47,60.96,71.12,31.751466,58.967008,10.0,13.0,...,Eager to Please,0.6,Alert/Responsive,Loyal,66.040,45.359237,11.5,Tall,Heavy,Long
4,Alaskan Malamute,The Alaskan Malamute stands 23 to 25 inches at...,"Affectionate, Loyal, Playful",58,58.42,63.50,34.019428,38.555351,10.0,14.0,...,Independent,0.8,Friendly,Playful,60.960,36.287390,12.0,Tall,Heavy,Long


In [77]:
dataframe.isna().sum()

breed                          0
description                    0
temperament                    0
popularity                     0
min_height                     0
max_height                     0
min_weight                     0
max_weight                     0
min_expectancy                 0
max_expectancy                 0
group                          0
grooming_frequency_value       0
grooming_frequency_category    0
shedding_value                 0
shedding_category              0
energy_level_value             0
energy_level_category          0
trainability_value             0
trainability_category          0
demeanor_value                 0
demeanor_category              0
temperament_category           0
height                         0
weight                         0
expectancy                     0
height_category                0
weight_category                0
expectancy_category            0
dtype: int64

In [16]:
dataframe['popularity'].value_counts()

140    2
of     2
17     2
7      2
133    2
      ..
45     1
123    1
54     1
169    1
184    1
Name: popularity, Length: 183, dtype: int64

In [18]:
invalid_rows = dataframe[dataframe['popularity'] == 'of']
print(invalid_rows)

      breed                                        description  \
14  Azawakh  This ancient hunting hound is so lean and rang...   
15   Barbet  The defining characteristic of this rustic, me...   

                                temperament popularity  min_height  \
14  Loyal, Independent, Deeply Affectionate         of       59.69   
15          Friendly, Bright, Sweet-Natured         of       48.26   

    max_height  min_weight  max_weight  min_expectancy  max_expectancy  ...  \
14       73.66   14.968548   24.947580            12.0            15.0  ...   
15       62.23   15.875733   29.483504            12.0            14.0  ...   

   trainability_category  demeanor_value demeanor_category  \
14           Independent             0.8          Friendly   
15             Agreeable             0.8          Friendly   

    temperament_category  height     weight expectancy  height_category  \
14                 Loyal  66.675  19.958064       13.5             Tall   
15             

In [19]:
# Mengganti nilai 'of' dengan nilai yang valid
dataframe.loc[dataframe['popularity'] == 'of', 'popularity'] = 50

In [20]:
invalid_rows = dataframe[dataframe['popularity'] == 'of']
print(invalid_rows)

Empty DataFrame
Columns: [breed, description, temperament, popularity, min_height, max_height, min_weight, max_weight, min_expectancy, max_expectancy, group, grooming_frequency_value, grooming_frequency_category, shedding_value, shedding_category, energy_level_value, energy_level_category, trainability_value, trainability_category, demeanor_value, demeanor_category, temperament_category, height, weight, expectancy, height_category, weight_category, expectancy_category]
Index: []

[0 rows x 28 columns]


In [37]:
dataframe['popularity'] = dataframe['popularity'].astype(int)
dataframe.to_csv('./data/akc-data-latest-v4.csv', index=False)

In [24]:
dataframe.dtypes

breed                           object
description                     object
temperament                     object
popularity                       int32
min_height                     float64
max_height                     float64
min_weight                     float64
max_weight                     float64
min_expectancy                 float64
max_expectancy                 float64
group                           object
grooming_frequency_value       float64
grooming_frequency_category     object
shedding_value                 float64
shedding_category               object
energy_level_value             float64
energy_level_category           object
trainability_value             float64
trainability_category           object
demeanor_value                 float64
demeanor_category               object
temperament_category            object
height                         float64
weight                         float64
expectancy                     float64
height_category          

In [66]:

# Kolom numerik
numerical_cols = [ 'popularity', 'min_height', 'max_height', 'min_weight', 'max_weight', 'min_expectancy', 'max_expectancy']
dataframe_numeric = dataframe[numerical_cols]

# Mengubah data kategorikal menjadi numerik
categorical_cols = ['expectancy_category', 'height_category', 'weight_category', 'demeanor_category', 'trainability_category', 'energy_level_category', 'shedding_category', 'grooming_frequency_category', 'temperament_category']
dataframe_categorical = dataframe[categorical_cols].astype('category')

In [55]:
dataframe_encoded = pd.get_dummies(dataframe_categorical)

# Menggabungkan data numerik dan data encoded
data = pd.concat([dataframe_numeric, dataframe_encoded], axis=1)
pd.DataFrame(data)

,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,expectancy_category_Long,expectancy_category_Medium,expectancy_category_Short,...,temperament_category_Active,temperament_category_Amiable,temperament_category_Brave,temperament_category_Confident,temperament_category_Gentle,temperament_category_Intelligent,temperament_category_Low-Key,temperament_category_Loyal,temperament_category_Other,temperament_category_Playful
0,148,22.86,29.21,3.175147,4.535924,12.0,15.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,113,63.50,68.58,22.679619,27.215542,12.0,15.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,60,58.42,58.42,22.679619,31.751466,11.0,14.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,47,60.96,71.12,31.751466,58.967008,10.0,13.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,58,58.42,63.50,34.019428,38.555351,10.0,14.0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,101,38.10,38.10,6.803886,8.164663,12.0,15.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
184,65,50.80,60.96,15.875733,31.751466,12.0,15.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
185,167,54.61,63.50,20.411657,29.483504,12.0,14.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
186,140,25.40,58.42,4.535924,24.947580,13.0,18.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [58]:
# Menghapus baris dengan nilai non-numerik
data = data.dropna()

# Menentukan fitur (features) dan target variabel
features = data
target = dataframe['breed']

# Menghitung similarity matrix menggunakan cosine similarity
similarity_matrix = cosine_similarity(features)

# Inisialisasi model KNN
knn_model = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_model.fit(features)

NearestNeighbors(metric='cosine')

In [68]:
query_breed = 'Afghan Hound'
query_index = target[target == query_breed].index[0]
print(query_index)

1


In [44]:
distances, indices = knn_model.kneighbors(features.iloc[query_index].values.reshape(1, -1))


c:\Users\iqbal.DESKTOP-4KFC9LN\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


In [45]:
print("Ras yang mirip dengan", query_breed, ":")
for index in indices[0]:
    recommended_breed = target.iloc[index]
    recommended_data = dataframe.loc[index]  # Mendapatkan baris data lengkap dari dataframe berdasarkan indeks
    similiarity_score = similarity_matrix[query_index][index]
    print("Ras Anjing:", recommended_breed, "(Score: %.4f)" % similiarity_score)
    print("Data Karakteristik:")
    print(recommended_data)
    print()


Ras yang mirip dengan Afghan Hound :
Ras Anjing: Afghan Hound (Score: 1.0000)
Data Karakteristik:
breed                                                               Afghan Hound
description                    The Afghan Hound is an ancient breed, his whol...
temperament                            Dignified, Profoundly Loyal, Aristocratic
popularity                                                                   113
min_height                                                                  63.5
max_height                                                                 68.58
min_weight                                                               22.6796
max_weight                                                               27.2155
min_expectancy                                                                12
max_expectancy                                                                15
group                                                                Hound Group
grooming_fr

In [75]:
# Menyimpan model KNN
dump(knn_model, 'knn_model.pkl')

['knn_model.pkl']